In [5]:
from pathlib import Path
from collections import Counter

import spacy
from spacy import displacy
from matplotlib import pyplot as plt
from joblib import Parallel, delayed

from src.gen.util import read_gzip_data, write_gzip_data

datap = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/data/level/2")
dumpp = Path("/users/k21190024/study/fact-check-transfer-learning/scratch/dumps/explore/level/2")

nlp = spacy.load("en_core_web_lg")

# Load data

In [4]:
cla = "claims_lemma.pkl.gz"
cor = "corpus_lemma.pkl.gz"

sf_claims = read_gzip_data(datap.joinpath("scifact", cla))
sf_corpus = read_gzip_data(datap.joinpath("scifact", cor))

f_claims = read_gzip_data(datap.joinpath("fever", cla))
f_corpus = read_gzip_data(datap.joinpath("fever", cor))

cf_claims = read_gzip_data(datap.joinpath("climatefever", cla))
cf_corpus = read_gzip_data(datap.joinpath("climatefever", cor))

# Explore

## Most common words